# Indexing in MongoDB:

## Single-field indexes:

Let's evaluate the two situations -where we used index and the one in which we don't- performance wise.

Let's query the restaurant with: 41156888 as id.

In [3]:
import pymongo as pg #Among other things handles conncection between the db and python environement 
from pprint import pprint #pretty print
import datetime #Needed to format the datetime input later on.
import pandas as pd

In [4]:
client=pg.MongoClient()
db=client.test

In [5]:
db.restaurants.find_one()

{'_id': ObjectId('5cce307bd398404a2c2a70da'),
 'address': {'building': '469',
  'coord': [-73.961704, 40.662942],
  'street': 'Flatbush Avenue',
  'zipcode': '11225'},
 'borough': 'Brooklyn',
 'cuisine': 'Hamburgers',
 'grades': [{'date': datetime.datetime(2014, 12, 30, 0, 0),
   'grade': 'A',
   'score': 8},
  {'date': datetime.datetime(2014, 7, 1, 0, 0), 'grade': 'B', 'score': 23},
  {'date': datetime.datetime(2013, 4, 30, 0, 0), 'grade': 'A', 'score': 12},
  {'date': datetime.datetime(2012, 5, 8, 0, 0), 'grade': 'A', 'score': 12}],
 'name': "Wendy'S",
 'restaurant_id': '30112340'}

In [101]:
print("Straight search")
res=db.restaurants.find({'restaurant_id':'41156888'}).explain()
pd.DataFrame({'Examined documents':[res['executionStats']['totalDocsExamined']],'Time to execute(ms)':[res['executionStats']['executionTimeMillis']],'Num of results':[res['executionStats']['nReturned']]})

Straight search


,Examined documents,Num of results,Time to execute(ms)
0,25359,1,26


We examined all the files! (British museum search).

In [102]:
print('Using limit(1)')
limres=db.restaurants.find({'restaurant_id':'41156888'}).limit(1).explain()
pd.DataFrame({'Examined documents':[limres['executionStats']['totalDocsExamined']],'Time to execute(ms)':[limres['executionStats']['executionTimeMillis']],'Num of results':[limres['executionStats']['nReturned']]})

Using limit(1)


,Examined documents,Num of results,Time to execute(ms)
0,6027,1,10


There is a considerable decrease for both the number of examined files and the execution time. Although, this is not a very attractive way for tackling this problem of query performance in general, and that's because...well for starters, it's not always the case that we are looking only for one of the documents fulfilling the given condition. In this case since the restaurant id is unique, it's quite convenient, we get the same result as a full search with a lower cost...In fact, the research starts and once it founds a result matching, it stops so we get that sense of improved performance while it's only due to finding the wanted result sooner rather than later (It could have found it much later and the performance wouldn't be this good). More technically speaking, limit() determinates how many time we are going to iterate over the cursor (that contains the result of the query) in order to generate the returned value. If we are going to iterate only once, once a result is obtained continuing has no meaning. So it's improved performance but it's 'fake' improvement, in other words there is no heuristic (when it comes to limit()) that optimizes the search within the documents. General queries don't always get this convenient. 

Indexes are a great way to optimize queries like this because they organize data by a given field to let MongoDB find it

quickly. We will try to create an index on the retaurant_id field:

### Index creation:

In [103]:
db.restaurants.create_index([('restaurant_id',1)]) #Create index on the restaurant_id field.

'restaurant_id_1'

In [117]:
pprint(db.restaurants.index_information())

{'_id_': {'key': [('_id', 1)], 'ns': 'test.restaurants', 'v': 2},
 'cuisine_1': {'key': [('cuisine', 1)], 'ns': 'test.restaurants', 'v': 2}}


In [105]:
print("Indexed search:")
Indres=db.restaurants.find({'restaurant_id':'41156888'}).explain()
pd.DataFrame({'Examined documents':[Indres['executionStats']['totalDocsExamined']],'Time to execute(ms)':[Indres['executionStats']['executionTimeMillis']],'Num of results':[Indres['executionStats']['nReturned']]})

Indexed search:


,Examined documents,Num of results,Time to execute(ms)
0,1,1,2


Optimal performance! Thing to be expected. Although this gives great read peerformance although it hurts the insertion performances when we have a lot of indexes, since they too need update at each transaction. So the task of choosing fields to index is really important by keeping in mind this trade-off.

## Compound indexes:

When your query has multiple sort directions or multiple keys in the criteria, it is useful to use a compound index,
which is an index on more than one field. The order of fields listed in a compound index has significance. For
instance, if a compound index consists of { userid: 1, score: -1 }, the index sorts first by userid and
then, within each userid value, sorts by score.

In [106]:
db.restaurants.drop_index('restaurant_id_1') #Dropping the single index created earlier.

In [122]:
db.restaurants.create_index([('cuisine',1),('name',-1)])

'cuisine_1_name_-1'

In [125]:
IndCres=db.restaurants.find({'cuisine':'Moroccan'}).sort('name').explain()
pd.DataFrame({'Examined documents':[IndCres['executionStats']['totalDocsExamined']],'Time to execute(ms)':[IndCres['executionStats']['executionTimeMillis']],'Num of results':[IndCres['executionStats']['nReturned']]})

,Examined documents,Num of results,Time to execute(ms)
0,15,15,0


Optimal yet again! The search goes straight to the goal.

Let's query restaurants for which the cuisine is between: 'Pizza' & 'Mexican'.

In [139]:
result=db.restaurants.find({'$and':[{'cuisine':{'$gt':'Mexican'}},{'cuisine':{'$lt':'Pizza'}}]}).explain()
pd.DataFrame({'Examined documents':[result['executionStats']['totalDocsExamined']],'Time to execute(ms)':[result['executionStats']['executionTimeMillis']],'Num of results':[result['executionStats']['nReturned']]})

,Examined documents,Num of results,Time to execute(ms)
0,1334,1334,5


Optimal result!

In [142]:
result=db.restaurants.find({'$and':[{'cuisine':{'$gt':'Mexican'}},{'cuisine':{'$lt':'Pizza'}}]}).sort('name').explain()
pd.DataFrame({'Examined documents':[result['executionStats']['totalDocsExamined']],'Time to execute(ms)':[result['executionStats']['executionTimeMillis']],'Num of results':[result['executionStats']['nReturned']]})

,Examined documents,Num of results,Time to execute(ms)
0,1334,1334,15


Time performance decreased due to the sorting done after fetching unordered data (name wise).

Inversing the order of the compound index.

In [145]:
db.restaurants.create_index([('name',1),('cuisine',1)])

'name_1_cuisine_1'

In [148]:
resul=db.restaurants.find({'$and':[{'cuisine':{'$gt':'Mexican'}},{'cuisine':{'$lt':'Pizza'}}]}).hint('name_1_cuisine_1').sort('name').explain()
pd.DataFrame({'Examined documents':[resul['executionStats']['totalDocsExamined']],'Time to execute(ms)':[resul['executionStats']['executionTimeMillis']],'Num of results':[resul['executionStats']['nReturned']]})

,Examined documents,Num of results,Time to execute(ms)
0,25359,1334,87


## Aggregating data in MongoDB - Aggregation Pipeline:

Aggregation operations group values from multiple documents together, and can perform a variety of operations on the grouped data to return a single result. Aggregation in MongoDB can be performed using MongoDB’s aggregation framework or MapReduce.

### MongoDB aggregation Pipeline:

Each step of the pipeline maps to an aggregation framework operator. Each operator receives a stream of documents,
does some type of transformation on these documents, and then passes on the results of the transformation. If it
is the last pipeline operator, these results are returned to the client. Otherwise, the results are streamed to the next
operator as input. Operators can be combined in any order and repeated as many times as necessary.

In the db.collection.aggregate method, pipeline stages appear in an array. Documents pass through the stages in sequence.

### Pipeline Operations:

#### $match:

We generaly never not want to start our pipeline with this, since it will lighten the work mass done by the following pipeline operators. 

In [41]:
magr=db.restaurants.aggregate([{'$match':{'borough':'Manhattan'}}])

It is important to note that the results of an aggregation are only one single use. After using it once it is destroyed.

In [42]:
pprint(list(magr)[0:5])

[{'_id': ObjectId('5cce307bd398404a2c2a70db'),
  'address': {'building': '351',
              'coord': [-73.98513559999999, 40.7676919],
              'street': 'West   57 Street',
              'zipcode': '10019'},
  'borough': 'Manhattan',
  'cuisine': 'Irish',
  'grades': [{'date': datetime.datetime(2014, 9, 6, 0, 0),
              'grade': 'A',
              'score': 2},
             {'date': datetime.datetime(2013, 7, 22, 0, 0),
              'grade': 'A',
              'score': 11},
             {'date': datetime.datetime(2012, 7, 31, 0, 0),
              'grade': 'A',
              'score': 12},
             {'date': datetime.datetime(2011, 12, 29, 0, 0),
              'grade': 'A',
              'score': 12}],
  'name': 'Dj Reynolds Pub And Restaurant',
  'restaurant_id': '30191841'},
 {'_id': ObjectId('5cce307bd398404a2c2a70e6'),
  'address': {'building': '1',
              'coord': [-73.96926909999999, 40.7685235],
              'street': 'East   66 Street',
              'zi

#### $project:

Allows us to select only certain number of fields of interest, and also to rename fields. 
Let's add this to our original pipeline by only returning the name and grades of the restaurants and let's rename these two to respectively: restaurantName and restaurantGrades. 

In [66]:
mpagr=db.restaurants.aggregate([
    {'$match':{'borough':'Manhattan'}},
    {'$project':{'restaurantName':'$name','restaurantGrades':'$grades'}}])

In [50]:
pprint(list(mpagr)[0:5])

[{'_id': ObjectId('5cce307bd398404a2c2a70db'),
  'restaurantGrades': [{'date': datetime.datetime(2014, 9, 6, 0, 0),
                        'grade': 'A',
                        'score': 2},
                       {'date': datetime.datetime(2013, 7, 22, 0, 0),
                        'grade': 'A',
                        'score': 11},
                       {'date': datetime.datetime(2012, 7, 31, 0, 0),
                        'grade': 'A',
                        'score': 12},
                       {'date': datetime.datetime(2011, 12, 29, 0, 0),
                        'grade': 'A',
                        'score': 12}],
  'restaurantName': 'Dj Reynolds Pub And Restaurant'},
 {'_id': ObjectId('5cce307bd398404a2c2a70e6'),
  'restaurantGrades': [{'date': datetime.datetime(2014, 5, 7, 0, 0),
                        'grade': 'A',
                        'score': 3},
                       {'date': datetime.datetime(2013, 5, 3, 0, 0),
                        'grade': 'A',
                

Now we will use operators that helps deal with arithmetic expressions. 

Let's calculate for each restaurant the average of grades.

In [130]:
mpgagr=db.restaurants.aggregate([
    {'$match':{'borough':'Manhattan'}},
    {'$project':{'restaurantName':'$name','restaurantGrades':'$grades','avgGrade':{'$avg':'$grades.score'}}}])

In [131]:
pprint(list(mpgagr)[0:5])

[{'_id': ObjectId('5cce307bd398404a2c2a70db'),
  'avgGrade': 9.25,
  'restaurantGrades': [{'date': datetime.datetime(2014, 9, 6, 0, 0),
                        'grade': 'A',
                        'score': 2},
                       {'date': datetime.datetime(2013, 7, 22, 0, 0),
                        'grade': 'A',
                        'score': 11},
                       {'date': datetime.datetime(2012, 7, 31, 0, 0),
                        'grade': 'A',
                        'score': 12},
                       {'date': datetime.datetime(2011, 12, 29, 0, 0),
                        'grade': 'A',
                        'score': 12}],
  'restaurantName': 'Dj Reynolds Pub And Restaurant'},
 {'_id': ObjectId('5cce307bd398404a2c2a70e6'),
  'avgGrade': 3.25,
  'restaurantGrades': [{'date': datetime.datetime(2014, 5, 7, 0, 0),
                        'grade': 'A',
                        'score': 3},
                       {'date': datetime.datetime(2013, 5, 3, 0, 0),
              

In [98]:
mpgagr=db.restaurants.find({'grades.score':{'$gt':12}})

In [97]:
pprint(list(mpgagr)[0:5])

OperationFailure: $all needs an array